In [ ]:
import pandas as pd
import json
import csv
import cplex
import time

In [ ]:
def listar(x):
    rawData = []
    start = 0
    for i in range(len(x)):
        if x[i] == ' ' or x[i] == '\n':
            rawData.append(x[start:i])
            start = i + 1
    return rawData

def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end=" ")
        print()
        
def ler_grafo(u):
    arquive = open(u, 'r')
    content = arquive.read()
    rawData = []
    rawData = listar(content)

    vertices, arestas = [int(v) for v in rawData[:2]]
    edgeData = [json.loads(edge) for edge in rawData[2:]]
    print(str(vertices) + ' vertices, ' + str(arestas) + ' arestas, ')
    # print(edgeData)
    m1, m2 = [], []
    for i in range(vertices):
        m1.append([0]*vertices)
        m2.append([])
    
    for i in range(0,len(edgeData)-1, 2):
        m1[edgeData[i]][edgeData[i+1]] = 1
        m1[edgeData[i+1]][edgeData[i]] = 1

    if len(m1) != 0:
        for i in  range(len(m1)):
            for j in range(len(m1[i])):
                if m1[i][j] == 1:
                    m2[i].append(j+1)
    # print_matrix(m1)
    # print()
    # print_matrix(m2)
    return m2,vertices,arestas



In [ ]:
def resolve_modelo(m2,vertices,arestas,k, timethresh):
    
    cpx = cplex.Cplex()

    y = cpx.variables.add(obj=[0] * vertices,
                             lb=[0] * vertices, ub=[1] * vertices,
                             types=['B'] * vertices,
                             names=['y_%d' % (p) for p in range(1,vertices+1)])
    h = cpx.variables.add(obj=[1] * vertices,
                            lb=[0] * vertices, ub=[1] * vertices,
                            types=['B'] * vertices,
                            names=['h_%d' % (z) for z in range(1,vertices+1)])

    #Restrição 2
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([y[i] for i in range(vertices)], [1.0]*vertices)],
        senses=['E'],
        rhs= [k])


    #Restrição 3
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]], [1.0,-1.0])
                  for i in range(vertices)],
        senses=['L'] * vertices,
        rhs=[0.0] * vertices)
    
    
    #Restrição 4
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[j-1]], [1.0,-1.0])
                  for i in range(vertices) for j in m2[i]],
        senses=['L'] * (arestas*2),
        rhs=[0.0] * (arestas*2))
    
    
    #Restrição 5
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]]+[y[j-1] for j in m2[i]], [1.0,-1.0]+[-1.0]*len(m2[i]))
                  for i in range(vertices)],
        senses=['G'] * vertices,
        rhs=[-len(m2[i]) for i in range(vertices)])

    
    cpx.parameters.threads.set(1)
    cpx.parameters.timelimit.set(timethresh)
    cpx.objective.set_sense(cpx.objective.sense.maximize)
    
    cpx.write('model.lp')
    start_time = time.time()
    cpx.solve()
    my_time = time.time() - start_time
    status = cpx.solution.get_status()
    # if status != 101:
    #     relaxation = [cpx.solution.MIP.get_best_objective(),
    #                     cpx.solution.MIP.get_cutoff,
    #                     # cpx.solution.MIP.get_incubent_node(),
    #                     # cpx.solution.MIP.get_mip_relative_gap(),
    #                     cpx.solution.MIP.get_num_cuts(cpx.solution.MIP.cut_type.cover),
    #                     cpx.solution.MIP.get_subproblem_status()]
    # else:
    relaxation = []

    return status, cpx.solution.get_objective_value(), cpx.solution.MIP.get_best_objective(), ','.join(relaxation), cpx.solution.get_objective_value(), my_time


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 900
for log_n in range(2,15):
    n = 2**log_n
    for graphClass in ["star", "cubic", "bipartite", "BA", "lollipop"]:
        for k in range(1,4):
            k = 2**(log_n + k - 4)
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')

                index += 1
            results.to_csv(f'./results4.csv')
print("Fim.")


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = 0
time_threshold = 2800
if deuruim < 4:
    for log_n in range(2,15):
        n = 2**log_n
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'graphs/BA/BA_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            m2,v,e = ler_grafo(filename)
            status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
            if status != 101:
                deuruim += 1
            results.loc[index] = [f'{filename}_{k}', "BA", n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
            print(f'------------------------>{filename}_k:{k} feito.')
            index += 1
            results.to_csv(f'./results5.csv')
print("Fim.")


In [10]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 3600
for graphClass in ["BA", "lollipop"]:
    for log_n in range(2,15):
        n = 2**log_n
        kStep = int(n/20)
        if kStep == 0:
            kStep = 1
        for k in range(0,n,kStep):
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')
                index += 1
                results.to_csv(f'./results7.csv')
print("Fim.")


Default row names c1, c2 ... being created.


Começando graphs/star/star_4.txt_k:0.------------------------>
4 vertices, 3 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 15 rows and 8 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)


Default row names c1, c2 ... being created.


------------------------>graphs/star/star_4.txt_k:0 feito.
Começando graphs/star/star_4.txt_k:1.------------------------>
4 vertices, 3 arestas, 
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 4 times.
MIP Presolve eliminated 10 rows and 4 columns.
MIP Presolve modified 9 coefficients.
Aggregator did 4 substitutions.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.05 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.05 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.05 ticks)


Default row names c1, c2 ... being created.


...

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Reduced MIP has 526337 rows, 2048 columns, and 1577984 nonzeros.
Reduced MIP has 2048 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 6.97 sec. (3052.95 ticks)
Found incumbent of value 508.000000 after 7.30 sec. (3248.21 ticks)
Probing time = 0.04 sec. (5.16 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 50.90 sec. (10000.30 ticks)
Elapsed time for symmetry detection = 104.51 sec. (20000.40 ticks)
Found 2.418309e+2333 symmetric permutations.
Reduced MIP has 526337 rows, 2048 columns, and 1577984 nonzeros.
Reduced MIP has 2048 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 114.72 sec. (23464.13 ticks)
Probing time = 0.05 sec. (5.16 ticks)
Clique table members: 525312.
MIP empha

Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Found incumbent of value 0.000000 after 0.01 sec. (12.13 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2101249 rows and 4096 columns.
All rows and columns eliminated.
Presolve time = 1.63 sec. (800.86 ticks)

Root node processing (before b&c):
  Real time             =    1.68 sec. (839.19 ticks)
Sequential b&c:
  Real time             =    0.00 sec. (0.00 ticks)
                          ------------
Total (root+branch&cut) =    1.68 sec. (839.19 ticks)
------------------------>graphs/bipartite/bipartite_2048.txt_k:0 feito.
Começando graphs/bipartite/bipartite_2048.txt_k:102.------------------------>
2048 vertices, 1048576 arestas, 


Default row names c1, c2 ... being created.


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_TimeLimit                               3600
Tried aggregator 1 time.
Presolve has eliminated 0 rows and 0 columns...
Reduced MIP has 2101249 rows, 4096 columns, and 6301696 nonzeros.
Reduced MIP has 4096 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 52.49 sec. (14188.09 ticks)
Found incumbent of value 0.000000 after 54.70 sec. (14952.59 ticks)
Probing time = 0.18 sec. (12.37 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Elapsed time for symmetry detection = 88.93 sec. (10012.67 ticks)
Elapsed time for symmetry detection = 197.81 sec. (20013.37 ticks)
Elapsed time for symmetry detection = 301.47 sec. (30014.21 ticks)
Elapsed time for symmetry detection = 401.42 sec. (40031.39 ticks)
Elapsed time for symmetry detection = 499.93 sec. (50031.39 ticks)
Elapsed time for symmetry detection = 596.80 sec

In [ ]:
results

In [ ]:
results